In [9]:
import numpy as np
import edf
from time import time
import sys
import matplotlib.pyplot as plt

data = np.load('./c10_data/train.npz')
t_imgs = np.float32(data['imgs'])/255.

# Reshape the train image data to (idx, h, w, channel)
t_imgs = t_imgs.reshape(50000, 32, 32, 3)
t_labels = np.float32(data['labels'])

data = np.load('./c10_data/test.npz')
v_imgs = np.float32(data['imgs'])/255.

# Reshape the valid image data to (idx, h, w, channel)
v_imgs = v_imgs.reshape(10000, 32, 32, 3)
v_labels = np.float32(data['labels'])

In [10]:
import numpy as np

a = np.tile(np.arange(200), 3).reshape(3, 10, 10, 2)
x = np.arange(9, -1, -1).reshape(-1, 5)
y = np.arange(10).reshape(-1,5)
z = np.repeat(np.zeros(1, dtype=np.int), 10).reshape(-1, 5)
index = np.hstack((z, x, y)).reshape(-1, 3)
f = np.arange(36).reshape(3,3,2,2)

#index
#a[z,x,y]

In [11]:
def im2col(x, f, stride=1, wpad=0, hpad=0):
    #x: input
    #f: filter
    
    b, h, w, c = x.shape
    h_filter, w_filter, c1, c2 = f.shape
    #print(x)
    x_padded = np.pad(x, ((0,0), (hpad, hpad), (wpad, wpad), (0,0)), "constant", constant_values=0)
    
    out_w = (w + 2*wpad - w_filter) / stride + 1
    out_h = (h + 2*hpad - h_filter) / stride + 1
    
    # first get all height indices
    # both i and j are two dimensional matrix
    i1 = np.arange(h_filter).repeat(w_filter)
    i1 = np.tile(i1, c)
    i1 = i1.reshape(-1, 1)
    #print(i1.shape)
    i2 = np.tile(np.arange(out_w), out_h) * stride
    i2 = i2.reshape(1, -1).astype(int)
    #print(i2.shape)
    i = i1 + i2
    
    # calculate all indices for y axis
    j1 = np.tile(np.arange(w_filter), h_filter * c).reshape(-1,1)
    j2 = np.tile(np.arange(out_w), out_h) * stride
    j = j1 + j2.reshape(1, -1).astype(int)
    
    # calculate for the z axis
    k = np.repeat(np.arange(c), h_filter * w_filter).reshape(-1, 1)
    
    #print(i.shape)
    #print(j.shape)
    #print(k.shape)

    # i, j, k represents the indices of x, y, z dimensions correspondingly
    #print(i1.dtype)
    #print(j1.dtype)
    #print(k.dtype)
    cols = x_padded[:, i, j, k]
    # this returns us a matrix of x[batch][h_filter * w_filter * channel][position]
    
    #cols = cols.transpose(1,2,0).reshape(h_filter * w_filter * c, -1)
    #the matrix is then transposed to x[h_filter * w_filter][position][batch]
    
    return cols
    
    #return x_padded

pad = np.asarray([2,2])
#a
cols = im2col(a, f, 1, 2,2)
cols.shape
    

/Users/zeweichu/anaconda3/lib/python3.5/site-packages/numpy/lib/shape_base.py:873: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  return c.reshape(shape_out)


(3, 18, 144)

In [12]:

########################################### Convolution layer#############################################
############################### Please implement the forward abd backward method in this class ############## 
class Conv:

    def __init__(self,x,f,stride=1,pad=0):
        edf.components.append(self)
        self.x = x #image
        self.f = f #filter 
        pad = np.array(pad)
        if pad.shape == ():
            self.wpad = self.hpad = pad
        else:
            self.wpad = pad[0]
            self.hpad = pad[1]
            
        self.stride=stride
        self.grad = None if x.grad is None and f.grad is None else edf.DT(0) 

    ####################### Please implement this function####################### 
    def forward(self):
        #(filter_width * filter_height * channels, out_width * out_height * batch)
        
        b, h, w, c = self.x.shape
        h_filter, w_filter, c1, c2 = self.f.shape
        
        # this returns us a matrix of x[batch][h_filter * w_filter * channel][position]
        cols = im2col(self.x, self.f, self.stride, self.xpad, self.ypad)
        
        # channel2, h_fitler, w_filter, channel1
        f = self.f.transpose(3, 0, 1, 2)
        #channel2, h_filter * w_filter * channel1
        f = f.reshape(c2, -1)
        
        # out: c2, batch, position
        out = np.dot(f, cols)
        
        out_w = (w + 2*self.wpad - w_filter) / self.stride + 1
        out_h = (h + 2*self.hpad - h_filter) / self.stride + 1
        
        out = out.reshape(c2, b, out_h, out_w)
        out = out.transpose(1, 2, 3, 0)
        return out
    

    ####################### Please implement this function#######################         
    def backward(self):
        if self.x.grad is not None:
            pass
        if self.f.grad is not None:
            pass
        b, h, w, c = self.grad.shape
        

########################################### MaxPool layer#############################################
############################### Please implement the forward abd backward method in this class ##############             
class MaxPool:
    def __init__(self,x,ksz=2,stride=None):
        edf.components.append(self)
        self.x = x
        self.ksz=ksz
        if stride is None:
            self.stride=ksz
        else:
            self.stride=stride
        self.grad = None if x.grad is None else edf.DT(0)

    ####################### Please implement this function#######################     
    def forward(self):
        pass

    ####################### Please implement this function#######################             
    def backward(self):
        pass

                            
########################################### AvePool layer#############################################
############################### Please implement the forward abd backward method in this class ##############                             
class AvePool:
    def __init__(self,x,ksz=2,stride=None):
        edf.components.append(self)
        self.x = x
        self.ksz=ksz
        if stride is None:
            self.stride=ksz
        else:
            self.stride=stride
        self.grad = None if x.grad is None else edf.DT(0)
        
    ####################### Please implement this function#######################   
    def forward(self):
        pass

    ####################### Please implement this function#######################    
    def backward(self):
        pass



AttributeError: 'numpy.ndarray' object has no attribute 'grad'

In [ ]:
# for repeatability
np.random.seed(0)

# Inputs
inp = edf.Value()
lab = edf.Value()


prev_channel = 3 # RGB channel 
########################## Simple Convolution Nerual Network Model for Cifar 10 ##################################
##################################################################################################################
# please implement your main cnn model here, as described by the homework, you can mimic the previous code


W1 = edf.Param(edf.xavier((28*28,128)))
B1 = edf.Param(np.zeros((128)))
W2 = edf.Param(edf.xavier((128,10)))
B2 = edf.Param(np.zeros((10)))





# the standard classification layer, which you don't need to modify
pred = edf.SoftMax(pred)
loss = edf.Mean(edf.LogLoss(edf.Aref(pred,lab)))
acc = edf.Accuracy(pred,lab)


################################################################################################################## 
# evaluation bucket
bucket = 100
def eval_train():    
    
    # we only choose 1/5 of the train images for evaluation since evaluation the whole images is time consuming
    eval_imgs = t_imgs[::5]
    eval_labels = t_labels[::5]
    avg_acc = 0
    avg_loss = 0
    
    for seq in range(bucket):
        inp.set(eval_imgs[seq::bucket])
        lab.set(eval_labels[seq::bucket])
        edf.Forward()
        avg_acc += acc.value
        avg_loss += loss.value
    
    return avg_acc/bucket, avg_loss/bucket
        
def eval_test():
    
    avg_acc = 0
    avg_loss = 0
    for seq in range(bucket):
        inp.set(v_imgs[seq::bucket])
        lab.set(v_labels[seq::bucket])
        edf.Forward()
        avg_acc += acc.value
        avg_loss += loss.value
    
    return avg_acc/bucket, avg_loss/bucket

# initial accuracy 
random_acc, random_loss = eval_test()
print("Random test loss = %.4f, accuracy = %.4f" % (random_loss, random_acc))


################################################# train loop ######################################################
ep = 0
epoch = 10
batch = 100
train_loss = []; train_acc = []; test_loss =[]; test_acc = []
stime = time()
batches = range(0, len(t_labels), batch)

while ep < epoch:

    # randon shuffle the train data in each epoch
    perm = np.random.permutation(len(t_labels))

    for k in batches:
        inp.set(t_imgs[perm[k:k+batch]])
        lab.set(t_labels[perm[k:k+batch]])
        edf.Forward()
        edf.Backward(loss)
        edf.Adam()
        
    # evaluate on trainset
    t_acc, t_loss = eval_train()
    print("Epoch %d: train loss = %.4f [%.3f secs]" % (ep, t_loss,time()-stime))
    train_loss.append(t_loss)
    train_acc.append(t_acc)

    # evaluate on testset
    v_acc, v_loss = eval_test()
    print("test accuracy = %.4f" % v_acc)
    test_loss.append(v_loss)
    test_acc.append(v_acc)
    stime = time()
    ep += 1      


# plot
plt.figure(1)
plt.xlabel("epochs")
plt.ylabel("loss")
plt.plot(np.arange(len(test_loss)), test_loss, color='red')
plt.plot(np.arange(len(train_loss)), train_loss, color='blue')
plt.legend(['test loss', 'train loss'], loc='upper right')
plt.show()

plt.figure(2)
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.plot(np.arange(len(test_acc)), test_acc, color='red')
plt.plot(np.arange(len(train_acc)), train_acc, color='blue')
plt.legend(['test acc', 'train acc'], loc='lower right')
plt.show()